In [1]:
from datasets import Dataset
import pandas as pd
from determined.experimental import Determined
from determined.experimental import client
from determined import pytorch

# remove warnings
import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Set up .detignore file so the checkpoints directory is not packaged into future experiments
!echo checkpoints > .detignore

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_name = "tokyotech-llm/Swallow-13b-instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

## Inference test on the original HuggingFace model

In [8]:
def answer_me(model, messages, device, is_peft=False, det_expid=None):
    
    # The model loaded from a Determined checkpoint is finetuned using PEFT
    if det_expid and is_peft:
        checkpoint = client.get_experiment(experiment_id).top_checkpoint(sort_by="eval_loss", smaller_is_better=True)
        path = checkpoint.download()
        model_path = path+'/checkpoint-180'
        model = PeftModel.from_pretrained(model, model_path, torch_dtype=torch.bfloat16, device_map="auto")
        
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

    model_inputs = encodeds.to(device)
    model.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    print(decoded[0])

In [9]:
messages = [
    {"role": "system", "content": "あなたは誠実で優秀な日本人のアシスタントです。"},
    {"role": "user", "content": "東京工業大学の主なキャンパスについて教えてください"}
]

answer_me(model, messages, device, is_peft=False, det_expid=None)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

東京工業大学の主なキャンパスについて教えてください [/INST] 東京工業大学は、日本の東京都にある国立大学です。主なキャンパスは、大岡山キャンパスとすずかけ台キャンパスの deux です。

1. 大岡山キャンパス:
大岡山キャンパスは、東京工業大学のメインキャンパスです。このキャンパスは、東京都目黒区大岡山にあり、JR 山手線の大岡山駅から徒歩 5 分です。大岡山キャンパスには、工学部、情報工学部、環境・社会学部、生命情報学部、理工学研究科などがあります。

2. すずかけ台キャンパス:
すずかけ台キャンパスは、東京都世田谷区すずかけ台にあり、東急田園都市線のすずかけ台駅から徒歩 10 分です。すずかけ台キャンパスには、物質理工学部、生命理工学部、情報理工学部、環境・社会理工学部、生命情報理工学研究科などがあります。

東京工業大学は、これら deux のキャンパス以外にも、いくつかの研究施設や教育機関があります。例えば、すずかけ台キャンパスには、東京工業大学の附属高校もあります。</s>


## Inference test on the original HuggingFace model for a math question

In [10]:
messages = [
    {"role": "system", "content": "あなたは誠実で優秀な日本人のアシスタントです。"},
    {"role": "user", "content": "What is the value of 'y' when x = 3 in the equation y = 2x + 1?"}
]

answer_me(model, messages, device, is_peft=False, det_expid=None)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

What is the value of 'y' when x = 3 in the equation y = 2x + 1? [/INST] 3

The value of 'y' when x = 3 in the equation y = 2x + 1 is 3.

Here's the step-by-step explanation:

1. Plug in the value of x: When x = 3, the equation becomes y = 2(3) + 1.
2. Perform the multiplication: Multiplying 2 and 3, we get 6.
3. Add 1 to the result: So, the equation becomes y = 6 + 1.
4. Simplify the equation: The equation becomes y = 7.
5. The value of 'y' when x = 3 is 7.</s>


## Inference test on the finetuned model

In [11]:
experiment_id = 12034
messages = [
        {"role": "system", "content": "あなたは誠実で優秀な日本人のアシスタントです。"},
        {"role": "user", "content": "What is the value of 'y' when x = 3 in the equation y = 2x + 1?."}
    ]
answer_me(model, messages, device, is_peft=True, det_expid=experiment_id)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

What is the value of 'y' when x = 3 in the equation y = 2x + 1?. [/INST] 3 is the value of x when y = 2x + 1 is solved for y.</s>
